In [1]:
!pip install torch torchvision torchaudio
!pip install monai
!pip install numpy
!pip install matplotlib
!pip install itk

  Using cached filelock-3.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pillow-10.2.0-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.6 MB 667.8 kB/s eta 0:04:58
   ---------------------------------------- 0.2/198.6 MB 2.0 MB/s eta 0:01:41
   ---------------------------------------- 0.3/198.6 MB 2.5 MB/s eta 0:01:21
   ---------------------------------------- 0.3/198.6 MB 2.5 MB/s eta 0:01:21
   ----

In [2]:
import torch
from torch import nn
import monai
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class Sobel(nn.Module):
    def __init__(self):
        super().__init__()
        self.filter = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=0, bias=False)

        Gx = torch.tensor([[2.0, 0.0, -2.0], [4.0, 0.0, -4.0], [2.0, 0.0, -2.0]])
        Gy = torch.tensor([[2.0, 4.0, 2.0], [0.0, 0.0, 0.0], [-2.0, -4.0, -2.0]])
        G = torch.cat([Gx.unsqueeze(0), Gy.unsqueeze(0)], 0)
        G = G.unsqueeze(1)
        self.filter.weight = nn.Parameter(G, requires_grad=False)

    def forward(self, img):
        x = self.filter(img)
        x = torch.mul(x, x)
        x = torch.sum(x, dim=1, keepdim=True)
        x = torch.sqrt(x)
        return x


In [3]:
img_path = "Crete_no7_20um_1821x1671x1455_16bit_trans_376.nii.gz"
load_image = monai.transforms.LoadImage(dtype=np.float32, ensure_channel_first=True)
img = load_image(img_path)
print(img.shape)

torch.Size([1, 140, 140, 140])


In [5]:
# extract slices from original data
arr_orig = np.array(img[0])
middle_slice_orig_x = arr_orig[int(len(arr_orig) / 2), :, :]
middle_slice_orig_y = arr_orig[:, int(len(arr_orig) / 2), :]
middle_slice_orig_z = arr_orig[:, :, int(len(arr_orig) / 2)]

# create sobel gradient volume
img_sobel = Sobel().forward(img)
arr_sobel = np.array(img_sobel[0])
middle_slice_sobel_x = arr_sobel[int(len(arr_sobel) / 2), :, :]
middle_slice_sobel_y = arr_sobel[:, int(len(arr_sobel) / 2), :]
middle_slice_sobel_z = arr_sobel[:, :, int(len(arr_sobel) / 2)]

# visualize
fig, axes = plt.subplots(3, 2, figsize=(10, 10))
axes[0, 0].imshow(middle_slice_orig_x, cmap="gray")
axes[0, 0].set_title("Original Slices (x, y, z view)")
axes[0, 1].imshow(middle_slice_sobel_x, cmap="gray")
axes[0, 1].set_title("Sobel Slices (x, y, z view)")
axes[1, 0].imshow(middle_slice_orig_y, cmap="gray")
axes[1, 1].imshow(middle_slice_sobel_y, cmap="gray")
axes[2, 0].imshow(middle_slice_orig_z, cmap="gray")
axes[2, 1].imshow(middle_slice_sobel_z, cmap="gray")

RuntimeError: Given groups=1, weight of size [2, 1, 3, 3], expected input[1, 140, 140, 140] to have 1 channels, but got 140 channels instead